Cryptocurrency trading bot.

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# File:        cryptocurrency/crypto_logger_base.py
# By:          Samuel Duclos
# For          Myself
# Description: Simple Binance logger base class.

# Library imports.
from cryptocurrency.resample import resample
from binance.client import Client
from abc import abstractmethod, ABC
from time import sleep, time
from os.path import exists, join
from os import mkdir

import pandas as pd

class Crypto_logger_base(ABC):
    def __init__(self, interval='15s', delay=4.7, buffer_size=3000, directory='crypto_logs', 
                 log_name='crypto_log', raw=False):
        """
        :param interval: OHLCV interval to log. Default is 15 seconds.
        :param delay: delay between Binance API requests. Minimum calculated was 4.7 seconds.
        :param buffer_size: buffer size to avoid crashing on memory accesses.
        :param directory: the directory where to output the logs.
        :param log_name: name of the log file.
        :param raw: whether the log dumps raw (instantaneous) or OHLCV data.
        """
        self.interval = interval
        self.delay = delay
        self.buffer_size = buffer_size
        self.directory = directory
        self.raw = raw

        self.log_name = join(self.directory, log_name + '.txt')
        self.log_screened_name = join(self.directory, log_name + '_screened.txt')

        if not exists(self.directory):
            mkdir(self.directory)

    #self.get_from_file(log_name=self.log_name, from_raw=False)
    #self.get_from_file(log_name=self.input_log_name, from_raw=self.load_from_ohlcv)
    def get_from_file(self, log_name, from_raw=False):
        if from_raw:
            dataset = pd.read_csv(log_name, header=0, index_col=0)
        else:
            dataset = pd.read_csv(log_name, header=[0, 1], index_col=0)
        dataset.index = pd.DatetimeIndex(dataset.index)
        return dataset.sort_index(axis='index')

    @abstractmethod
    def get(self, **kwargs):
        raise NotImplementedError()

    @abstractmethod
    def screen(self, **kwargs):
        raise NotImplementedError()

    def put(self, dataset):
        dataset = dataset.copy().reset_index()
        if self.raw:
            dataset = dataset.drop_duplicates(subset=['symbol', 'count'], 
                                              keep='first', ignore_index=True)
        else:
            dataset = dataset.drop_duplicates(keep='last', ignore_index=True)

        if 'date' in dataset.columns:
            min_index_int = dataset[dataset['date'] == self.min_index].index[0]
            dataset = dataset.set_index('date')
        if not self.raw:
            dataset = resample(dataset, self.interval)
        if 'date' in dataset.columns:
            dataset = dataset.iloc[min_index_int:]

        dataset = dataset.tail(self.buffer_size)
        dataset.to_csv(self.log_name)
        self.min_index = dataset.index[0]
        return dataset

    def start(self, append=False, roll=0):
        """Main logger loop."""
        print('Starting crypto logger.')

        if exists(self.log_name) and 'output' in self.log_name:
            self.dataset = self.get_from_file(log_name=self.log_name, from_raw=False)
            self.dataset = self.dataset.tail(self.buffer_size)
        else:
            self.dataset = self.get()

        self.min_index = self.dataset.index[-1]
        self.dataset = self.put(self.dataset)

        while True:
            try:
                dataset = pd.concat([self.dataset, self.get()], axis='index', join='outer')
            except (KeyboardInterrupt, SystemExit):
                print('User terminated crypto logger process.')
                break
            except Exception as e:
                print(e)
            try:
                self.dataset = self.put(dataset)
            except (KeyboardInterrupt, SystemExit):
                print('Saving latest complete dataset...')
                self.dataset = self.put(dataset)
                print('User terminated crypto logger process.')
                break
            except Exception as e:
                print(e)
            try:
                if exists(self.log_screened_name):
                    dataset_screened_old = \
                        pd.read_csv(self.log_screened_name, index_col=0, header=0)
                else:
                    dataset_screened_old = None
                dataset_screened = self.screen(self.dataset)
                if dataset_screened is not None:
                    if roll != 0:
                        if append and exists(self.log_screened_name):
                            dataset_screened = \
                                pd.concat([dataset_screened_old, dataset_screened], axis='index')
                            dataset_screened = \
                                dataset_screened.drop_duplicates(subset=['symbol'], keep='last')
                        dataset_screened = dataset_screened.tail(roll)
                        dataset_screened.to_csv(self.log_screened_name)
                    elif append:
                        dataset_screened.to_csv(self.log_screened_name, mode='a')
                    else:
                        dataset_screened.to_csv(self.log_screened_name)
            except (KeyboardInterrupt, SystemExit):
                print('User terminated crypto logger process.')
                break
            except Exception as e:
                print(e)
            sleep(self.delay)
        print('Crypto logger process done.')

In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# File:        cryptocurrency/crypto_logger_input.py
# By:          Samuel Duclos
# For          Myself
# Description: Simple Binance logger circular buffered for N time precision.

# Library imports.
#from cryptocurrency.crypto_logger_base import Crypto_logger_base
from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.exchange import Cryptocurrency_exchange
from cryptocurrency.conversion import get_timezone_offset_in_seconds
from cryptocurrency.conversion_table import get_conversion_table, get_tradable_tickers_info
from os import mkdir
from os.path import exists, join

import datetime
import pandas as pd
pd.options.mode.chained_assignment = None

class Crypto_logger_input(Crypto_logger_base):
    def __init__(self, delay=4.7, interval='15s', buffer_size=3000, 
                 price_percent=5.0, volume_percent=0.0, as_pair=False):
        """
        :param interval: OHLCV interval to log. Default is 15 seconds.
        :param delay: delay between Binance API requests. Minimum calculated was 4.7 seconds.
        :param buffer_size: buffer size to avoid crashing on memory accesses.
        :param price_percent: price move percent.
        :param volume_percent: volume move percent.
        """
        self.resample = None
        self.price_percent = price_percent
        self.volume_percent = volume_percent
        self.as_pair = as_pair
        super().__init__(interval=interval, delay=delay, buffer_size=buffer_size, 
                         directory='crypto_logs', log_name='crypto_input_log_' + interval, 
                         raw=True)

        authenticator = Cryptocurrency_authenticator(use_keys=False, testnet=False)
        self.client = authenticator.spot_client

        exchange = Cryptocurrency_exchange(client=self.client, directory=self.directory)
        self.exchange_info = exchange.info

        self.offset_s = get_timezone_offset_in_seconds()

    def filter_movers(self, dataset, count=1000, price_percent=5.0, volume_percent=0.0):
        dataset = dataset.reset_index()
        dataset[['price_change_percent', 'rolling_quote_volume']] = \
            dataset[['price_change_percent', 'rolling_quote_volume']].astype(float)
        dataset['last_price_move'] = dataset['price_change_percent'].copy()
        dataset['last_volume_move'] = dataset['rolling_quote_volume'].copy()
        movers = dataset.groupby(['symbol'])
        dataset = dataset.drop(columns=['last_price_move', 'last_volume_move'])
        price_movers = movers['last_price_move']
        volume_movers = movers['last_volume_move']
        price_movers = price_movers.agg(lambda x: x.diff(1).abs().iloc[-1])
        volume_movers = volume_movers.agg(lambda x: (100 * x.pct_change(1)).iloc[-1])
        price_movers = price_movers.sort_values(ascending=False)
        volume_movers = volume_movers.sort_values(ascending=False)
        price_movers = price_movers[price_movers > 0.0]
        price_movers = price_movers.to_frame(name='last_price_move')
        volume_movers = volume_movers.to_frame(name='last_volume_move')
        movers = pd.concat([price_movers, volume_movers], axis='columns')
        movers = movers.reset_index()
        price_movers_mask = movers['last_price_move'] > price_percent
        volume_movers_mask = movers['last_volume_move'] > volume_percent
        movers = movers[price_movers_mask & volume_movers_mask]
        movers = movers.sort_values(by=['last_volume_move', 'last_price_move'], ascending=False)
        movers = movers.tail(count).reset_index(drop=True)
        return dataset.merge(right=movers, how='right', on=['symbol']).set_index('date')

    def screen(self, dataset):
        dataset = get_tradable_tickers_info(dataset, as_pair=self.as_pair)
        dataset = self.filter_movers(dataset, count=1000, 
                                     price_percent=self.price_percent, 
                                     volume_percent=self.volume_percent)
        return dataset.drop_duplicates(subset=['symbol', 'count'], keep='last')

    def prepare_downsampling(self, dataset):
        dataset['close_time'] /= 1000
        dataset['close_time'] = \
            dataset['close_time'].apply(datetime.datetime.fromtimestamp)
        dataset['date'] = pd.DatetimeIndex(dataset['close_time']).round(self.interval)
        return dataset.set_index('date').sort_index()

    def get(self):
        """Get all pairs data from Binance API."""
        dataset = get_conversion_table(self.client, self.exchange_info, offset_s=self.offset_s, 
                                       as_pair=self.as_pair, dump_raw=False)
        self.conversion_table = dataset.copy()
        #self.conversion_table.to_csv('crypto_logs/conversion_table.txt')
        return self.prepare_downsampling(self.conversion_table)

In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# File:        crypto_logger_input_1min.py
# By:          Samuel Duclos
# For          Myself
# Description: Simple Binance logger output for the 1 minute interval.

# Library imports.
#from cryptocurrency.crypto_logger_input import Crypto_logger_input

crypto_logger_input_1min = Crypto_logger_input(delay=12, interval='15s', buffer_size=3000, 
                                               price_percent=1.0, volume_percent=1.0, 
                                               as_pair=True)
crypto_logger_input_1min.start(append=True, roll=60)

Starting crypto logger.
User terminated crypto logger process.
Crypto logger process done.


In [4]:
import pandas as pd

crypto_input_log_1min = 'crypto_logs/crypto_input_log_1min.txt'
df = pd.read_csv(crypto_input_log_1min, header=0, index_col=0)
df.index = pd.to_datetime(df.index, utc=True)
df

,symbol,base_asset,quote_asset,is_shorted,price_change_percent,weighted_average_price,open,high,low,close,...,rolling_traded_volume,traded_bid_volume,traded_ask_volume,traded_price,traded_bid_price,traded_ask_price,bid_ask_change_percent,bid_ask_volume_percent,traded_bid_ask_change_percent,traded_bid_ask_volume_percent
date,,,,,,,,,,,,,,,,,,,,,
2022-11-10 19:57:00+00:00,BUSDVAI,BUSD,VAI,False,0.770,1.020354,1.03900,1.05100,1.01100,1.04700,...,1.398650e+10,9.500021e+08,4.598706e+09,482.121510,NaN,NaN,0.191022,0.297240,NaN,17.121140
2022-11-11 01:47:00+00:00,BDOTDOT,BDOT,DOT,False,0.231,0.736763,0.73680,0.74750,0.72980,0.73850,...,1.560469e+05,9.396900e+01,6.047088e+01,4.310000,3.180000,3.180000,0.135410,60.845036,0.000000,60.845036
2022-11-11 01:50:00+00:00,CKBBUSD,CKB,BUSD,False,2.941,0.002793,0.00272,0.00294,0.00259,0.00280,...,1.190554e+06,0.000000e+00,0.000000e+00,0.002810,0.000000,0.000000,0.709220,71.436818,NaN,NaN
2022-11-11 01:50:00+00:00,UMATRY,UMA,TRY,False,15.254,33.813563,29.50000,39.00000,29.40000,34.00000,...,1.981827e+06,1.568424e+03,7.353920e+04,1.778000,9.666395,9.695644,0.293255,0.005869,0.301665,2.088236
2022-11-11 01:50:00+00:00,TBUSD,T,BUSD,False,13.665,0.017587,0.01610,0.01880,0.01600,0.01830,...,6.059553e+05,6.779100e-01,5.813307e+01,0.018400,0.000300,0.000300,0.543478,17.232169,0.000000,1.152693
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-11 01:54:00+00:00,DOGEBUSD,DOGE,BUSD,False,16.982,0.085714,0.07496,0.09384,0.07437,0.08769,...,6.679959e+08,5.413160e+04,1.663081e+06,78.179900,inf,inf,0.011405,32.195343,NaN,3.152294
2022-11-11 01:54:00+00:00,DOGEUSDT,DOGE,USDT,False,17.104,0.085997,0.07513,0.09461,0.07451,0.08798,...,6.679959e+08,5.413160e+04,1.663081e+06,78.179900,inf,inf,0.022730,2.467924,NaN,3.152294
2022-11-11 01:54:00+00:00,MATICBUSD,MATIC,BUSD,False,28.646,1.021720,0.83920,1.16280,0.81810,1.07960,...,8.102626e+08,2.191924e+06,8.582029e+06,1.083200,84.283514,84.544048,0.037047,26.127228,0.308163,20.344658


In [5]:
import pandas as pd

crypto_input_log_15s = 'crypto_logs/crypto_input_log_15s.txt'
df = pd.read_csv(crypto_input_log_15s, header=0, index_col=0)
df.index = pd.to_datetime(df.index)
df

,symbol,base_asset,quote_asset,is_shorted,price_change_percent,weighted_average_price,open,high,low,close,...,rolling_traded_volume,traded_bid_volume,traded_ask_volume,traded_price,traded_bid_price,traded_ask_price,bid_ask_change_percent,bid_ask_volume_percent,traded_bid_ask_change_percent,traded_bid_ask_volume_percent
date,,,,,,,,,,,,,,,,,,,,,
2022-11-10 19:57:00,BUSDVAI,BUSD,VAI,False,0.770,1.020354,1.039000,1.051000,1.011000,1.047000,...,1.398701e+10,9.120073e+08,4.361288e+09,481.341660,NaN,NaN,0.191022,0.297240,NaN,17.294826
2022-11-11 01:46:45,BDOTDOT,BDOT,DOT,False,0.231,0.736763,0.736800,0.747500,0.729800,0.738500,...,1.564090e+05,9.396900e+01,6.066104e+01,4.320000,3.180000e+00,3.190000e+00,0.135410,60.845036,0.313480,60.770210
2022-11-11 01:50:15,TBUSD,T,BUSD,False,13.665,0.017587,0.016100,0.018800,0.016000,0.018300,...,6.059553e+05,6.779100e-01,5.813307e+01,0.018400,3.000000e-04,3.000000e-04,0.543478,17.232169,0.000000,1.152693
2022-11-11 01:50:45,TKOBUSD,TKO,BUSD,False,8.197,0.193712,0.183000,0.205000,0.181000,0.198000,...,7.735289e+05,9.851274e+03,3.044391e+05,0.198000,8.094730e+01,8.102441e+01,1.005025,4.195986,0.095169,3.134450
2022-11-11 01:52:15,TUSDT,T,USDT,False,14.286,0.017600,0.016100,0.019000,0.015800,0.018400,...,6.059553e+05,6.779100e-01,5.813307e+01,0.018400,3.000000e-04,3.000000e-04,0.540541,0.662186,0.000000,1.152693
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-11 01:56:45,LUNCUSDT,LUNC,USDT,False,5.181,0.000194,0.000185,0.000206,0.000182,0.000195,...,1.595716e+08,3.600848e-02,6.301496e-01,0.000195,3.000000e-08,3.000000e-08,0.030848,5.310885,0.000000,5.405395
2022-11-11 01:56:45,ETHBUSD,ETH,BUSD,False,11.447,1245.663480,1133.700000,1348.000000,1125.000000,1263.470000,...,4.575533e+09,2.760878e+09,1.594195e+10,1231.176787,NaN,NaN,0.000791,98.084581,NaN,14.761824
2022-11-11 01:56:45,APTBTC,APT,BTC,False,16.185,0.000273,0.000250,0.000310,0.000244,0.000291,...,1.832227e+08,6.773828e+04,6.038412e+05,5.028900,3.520586e+01,3.527405e+01,0.144529,30.166185,0.193307,10.086413


In [6]:
import pandas as pd

crypto_exchange_info = 'crypto_logs/crypto_exchange_info.txt'
exchange_info = pd.read_csv(crypto_exchange_info, header=0, index_col=0)
exchange_info

,symbol,base_asset,base_asset_precision,quote_asset,quote_precision,quote_asset_precision,base_asset_commission,quote_commission_precision,allow_trailing_stop,cancel_replace_allowed,min_price,max_price,tick_size,step_size,multiplier_up,multiplier_down
0,ETHBTC,ETH,8,BTC,8,8,8,8,True,True,1.000000e-06,922327.0,1.000000e-06,0.0001,5.0,0.2
1,LTCBTC,LTC,8,BTC,8,8,8,8,True,True,1.000000e-06,100000.0,1.000000e-06,0.0010,5.0,0.2
2,BNBBTC,BNB,8,BTC,8,8,8,8,True,True,1.000000e-06,100000.0,1.000000e-06,0.0010,5.0,0.2
3,NEOBTC,NEO,8,BTC,8,8,8,8,True,True,1.000000e-06,100000.0,1.000000e-06,0.0100,5.0,0.2
4,QTUMETH,QTUM,8,ETH,8,8,8,8,True,True,1.000000e-06,1000.0,1.000000e-06,0.1000,5.0,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1429,OSMOUSDT,OSMO,8,USDT,8,8,8,8,True,True,1.000000e-03,1000.0,1.000000e-03,0.0100,5.0,0.2
1430,OSMOBUSD,OSMO,8,BUSD,8,8,8,8,True,True,1.000000e-03,1000.0,1.000000e-03,0.0100,5.0,0.2
1431,HFTBTC,HFT,8,BTC,8,8,8,8,True,True,1.000000e-08,1.0,1.000000e-08,0.1000,5.0,0.2
1432,HFTBUSD,HFT,8,BUSD,8,8,8,8,True,True,1.000000e-04,1000.0,1.000000e-04,0.1000,5.0,0.2


In [7]:
from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.conversion_table import get_conversion_table, get_new_tickers
from cryptocurrency.exchange import Cryptocurrency_exchange
from cryptocurrency.conversion import convert_price

authenticator = Cryptocurrency_authenticator(use_keys=False, testnet=False)
client = authenticator.spot_client
exchange = Cryptocurrency_exchange(client=client, directory='crypto_logs')
exchange_info = exchange.info
conversion_table = get_conversion_table(client=client, exchange_info=exchange_info)
#assets = get_new_tickers(conversion_table=conversion_table)

symbol = 'BTCUSDT'
#original_size = exchange_info[exchange_info['symbol'] == symbol]['lastPrice']
quantity = df[df['symbol'] == symbol]['close'].iat[-1]
asset = exchange_info[exchange_info['symbol'] == symbol]['baseAsset'].iat[-1]

usdt_price = convert_price(size=1, from_asset=asset, to_asset='USDT', 
                           exchange_info=exchange_info, conversion_table=conversion_table)
usdt_price

KeyError: 'baseAsset'

In [ ]:
df.columns

In [ ]:
df = df[['symbol', 'lastPrice', 'volume', 'bidPrice', 'bidQty', 'askPrice', 'askQty']]
df = df.rename(columns={'lastPrice': 'close'})
df = df.pivot_table(index=['date'], columns=['symbol'], 
                    values=['close', 'volume', 'bidPrice', 'bidQty', 'askPrice', 'askQty'], 
                    aggfunc={'close': ['first', 'max', 'min', 'last'], 
                             'volume': 'last', 
                             'bidPrice': ['first', 'max', 'min', 'last'], 
                             'bidQty': 'sum', 
                             'askPrice': ['first', 'max', 'min', 'last'], 
                             'askQty': 'sum'})
df['volume'] = df['volume'].fillna(method='pad').fillna(0).diff(1)
df['bidQty']['sum'] = df['bidQty']['sum'].fillna(method='pad').fillna(0).diff(1)
df['askQty']['sum'] = df['askQty']['sum'].fillna(method='pad').fillna(0).diff(1)
df.columns = pd.MultiIndex.from_tuples([('_'.join(col[:2]), col[2]) for col in df.columns.values])
df = df.rename(columns={'close_first': 'open', 
                        'close_max': 'high', 
                        'close_min': 'low', 
                        'close_last': 'close', 
                        'volume_last': 'volume', 
                        'bidPrice_first': 'bid_open', 
                        'bidPrice_max': 'bid_high', 
                        'bidPrice_min': 'bid_low', 
                        'bidPrice_last': 'bid_close', 
                        'bidQty_sum': 'bid_volume', 
                        'askPrice_first': 'ask_open', 
                        'askPrice_max': 'ask_high', 
                        'askPrice_min': 'ask_low', 
                        'askPrice_last': 'ask_close', 
                        'askQty_sum': 'ask_volume'}, 
               level=0)
df

In [ ]:
df.columns

In [ ]:
df2.columns.get_level_values(1).unique()

In [ ]:
def resample_from_raw(df):
    df = df[['symbol', 'lastPrice', 'volume', 'quoteVolume']]
    df = df.rename(columns={'lastPrice': 'close', 
                            'volume': 'rolling_base_volume', 
                            'quoteVolume': 'rolling_quote_volume'})
    df = df.pivot_table(index=['date'], columns=['symbol'], 
                        values=['close', 'rolling_base_volume', 
                                'rolling_quote_volume'], 
                        aggfunc={'close': ['first', 'max', 'min', 'last'], 
                                 'rolling_base_volume': 'max', 
                                 'rolling_quote_volume': 'max'})
    df.columns = pd.MultiIndex.from_tuples([('_'.join(col[:2]), col[2]) for col in df.columns.values], 
                                           names=(None, 'symbol'))
    df = df.rename(columns={'close_first': 'open', 
                            'close_max': 'high', 
                            'close_min': 'low', 
                            'close_last': 'close', 
                            'rolling_base_volume_max': 'rolling_base_volume', 
                            'rolling_quote_volume_max': 'rolling_quote_volume'}, 
                   level=0)
    df['rolling_base_volume'] = df['rolling_base_volume'].fillna(method='pad')
    df['rolling_base_volume'].iloc[0] = 0
    df['rolling_quote_volume'] = df['rolling_quote_volume'].fillna(method='pad')
    df['rolling_quote_volume'].iloc[0] = 0
    df = df.sort_index().iloc[1:]
    df.columns = df.columns.swaplevel(0, 1)
    return df

df2 = resample_from_raw(df)
df2

In [ ]:
df2[(df2['NEBLBUSD']['volume'].diff() < 0)]['NEBLBUSD']['volume']

In [ ]:
#df2.columns = df2.columns.swaplevel(0, 1)
(~((df2[df2['volume'] > 0]['volume']).isna())).any().any()

In [ ]:
df2[(df2['volume'] > 0)]

In [ ]:
#%rm -rf crypto_logs